# Steven Ynoa 1-16-1865
## Actividad Semana 2

In [1]:
import re

# Para ejecutar el analizador solo debe de tener un entorno Python, si en dado caso no tiene Python.
# utilizar Google CoLab, no es necesario instalar solo correr el archivo.


# Analizador lexico para calculos matematicos basicos
TOKENS = [
    ('ID', r'[a-zA-Z_][a-zA-Z0-9_]*'),  #Variable ID
    ('EQUAL', r'='),  # Operador de asignación
    ('NUM', r'[0-9]+(\.[0-9]*)?'),  # Numeros enteros y decimales
    ('PLUS', r'\+'), # Signo de mas
    ('MINUS', r'\-'),# Signo de menos
    ('MUL', r'\*'),# Signo de multiplicacion
    ('DIV', r'/'),# Signo de division
    ('LPAREN', r'\('),# Abre parentesis
    ('RPAREN', r'\)'),# Cierra parentesis
    ('EXP', r'\^'),  # Exponenciación
    ('MOD', r'%'),  # Módulo
]

def LexerMatematico(code):
    # posicion inicial
    pos = 0
    # mientras recorre el codigo
    while pos < len(code):
        # revisa si los tokens son validos
        for token, regex in TOKENS:
            match = re.match(regex, code[pos:])
            # Si hay match agrupa el token con su valor, y continua con el ciclo
            if match:
                yield token, match.group(0)
                pos += match.end(0)
                break
        else:
            # si encuentra un valor desconocido, lanza una excepcion
            if code[pos] != ' ':
                raise Exception(f'Illegal character at position {pos}')
            # continua con el ciclo
            pos += 1

In [2]:
# Codigo a analizar
code = 'var suma = 2 + 3'

print(list(LexerMatematico(code)))

[('ID', 'var'), ('ID', 'suma'), ('EQUAL', '='), ('NUM', '2'), ('PLUS', '+'), ('NUM', '3')]


## Actividad Semana 3

In [3]:
class AnalizadorSintactico:
    # Inicializacion de la clase
    def __init__(self, tokens):
        # tokens del lexer
        self.tokens = list(tokens)
        # posicion 0
        self.pos = 0

    def inicializar(self):
        self.reglas_gramaticales()
        
    #parametros:
    # tipo: tipo de token 
    def verificar_pos(self, tipo):
        if self.pos >= len(self.tokens):
            raise Exception('se terminaron las entradas del token inesperadamente')
        # extrae el token
        token, _ = self.tokens[self.pos]

        if token != tipo:
            raise Exception(f'token inesperado: {token}, expected: {tipo}')
        
        self.pos += 1


    # como ejemplo verifica si los tokens forman una suma solamente
    def reglas_gramaticales(self):
        self.verificar_pos('NUM')
        while self.pos < len(self.tokens):
            self.verificar_pos('PLUS')
            self.verificar_pos('NUM')


In [4]:
# Prueba, si cambia el signo de + por otro signo, dara un error
code = '1 + 2 + 7'
tokens = list(LexerMatematico(code))
a_sintactico = AnalizadorSintactico(tokens)
a_sintactico.inicializar()


# Actividad semana 5

In [5]:
class AnalizadorSemantico:
    def __init__(self, tokens):
        self.tokens = tokens
        self.tabla_simbolos = {}

    def calcular(self):
        var, valor = self.tokens[0]
        self.tabla_simbolos[var] = int(self.tokens[2][1]) # Agregamos el primer número a la tabla de simbolos

        for i in range(3, len(self.tokens), 2): # Saltamos de dos en dos comenzando desde el cuarto elemento
            operador, _ = self.tokens[i]
            numero, valor = self.tokens[i+1]

            if operador == 'PLUS':
                self.tabla_simbolos[var] += int(valor)
            else:
                raise Exception(f'Operador inesperado: {operador}')
        return self.tabla_simbolos


In [6]:
analizador_sintactico = AnalizadorSintactico(tokens)
analizador_sintactico.inicializar() 

analizador_semantico = AnalizadorSemantico(tokens)
resultado = analizador_semantico.calcular()

print(f'Resultado:  {resultado}')

Resultado:  {'NUM': 9}


In [14]:
# Analizador sintactico
import ast

# las funciones que se encuentran vienen de la libreria ast, solo se reescribireron por eso el parametro de la clase es ast.NodeVisitor
class TranslateToJS(ast.NodeVisitor):
    # aqui se inicializan las variables
    def __init__(self):
        self.js_code = ""

    # aqui se recorre la lista de asignaciones
    def visit_Assign(self, node):
        if isinstance(node.value, ast.Num):
            self.js_code += f"let {node.targets[0].id} = {node.value.n};\n"
    
    # aqui se recorre las definiciones de las funciones y se sustituyen por 
    def visit_FunctionDef(self, node):
        # aqui recorre los parametros de la funcion
        args = ', '.join(arg.arg for arg in node.args.args)
        # aqui agrega el nombre de la funcion y sus parametros
        self.js_code += f"function {node.name}({args}) {{\n"
        self.generic_visit(node)
        # cierra funcion
        self.js_code += "}\n"

    # aqui se recorre las invocaciones o llamadas de las funciones o variables
    def visit_Call(self, node):
        # si el nodo es print, se sustituye por console.log
        if isinstance(node.func, ast.Name) and node.func.id == "print":
            self.js_code += f"console.log({node.args[0].id});\n"
    
    # aqui se empaqueta toda la logica
    def translate(self, python_code):
        python_ast = ast.parse(python_code)
        self.visit(python_ast)
        return self.js_code

translator = TranslateToJS()
python_code = """
a = 5


def primerTraductor():
    b=1
"""

js_code = translator.translate(python_code)
print(js_code)


let a = 5;
function primerTraductor() {
let b = 1;
}

